In [1]:
import tensorflow as tf
import numpy as np

In [2]:
x = tf.placeholder(tf.float32, [None, 2])
W = tf.Variable(tf.zeros([2, 5]))
b = tf.Variable(tf.zeros([5]))
y1 = tf.nn.sigmoid(tf.matmul(x, W) + b)
W1 = tf.Variable(tf.zeros([5, 2]))
b2 = tf.Variable(tf.zeros([2]))
y2 = tf.nn.sigmoid(tf.matmul(y1, W1) + b2)
y_ = tf.placeholder(tf.float32, [None, 2])
binary_entropylogloss = tf.reduce_mean(-tf.reduce_sum(y_ * tf.log(y2)+(1-y_)*tf.log(1-y2), reduction_indices=[1]))
train_steplogloss = tf.train.GradientDescentOptimizer(0.5).minimize(binary_entropylogloss)

squareloss = tf.reduce_mean(tf.reduce_sum(tf.square(y_ - y2), reduction_indices=[1]))
train_stepsquareloss = tf.train.GradientDescentOptimizer(0.5).minimize(squareloss)

In [3]:
ins=np.random.randint(0,2,(4000,2))
outs=np.array([[g[0]*g[1],(g[0]+g[1]>0)*1] for g in ins])

In [4]:
#VISUALIZATION --- This cell can be removed also if you don't want to visualize the graph
from IPython.display import clear_output, Image, display, HTML

sess = tf.InteractiveSession()
tf.global_variables_initializer().run()
def strip_consts(graph_def, max_const_size=32):
    """Strip large constant values from graph_def."""
    strip_def = tf.GraphDef()
    for n0 in graph_def.node:
        n = strip_def.node.add() 
        n.MergeFrom(n0)
        if n.op == 'Const':
            tensor = n.attr['value'].tensor
            size = len(tensor.tensor_content)
            if size > max_const_size:
                tensor.tensor_content = tf.compat.as_bytes("<stripped %d bytes>"%size)
    return strip_def
  
def show_graph(graph_def, max_const_size=32):
    """Visualize TensorFlow graph."""
    if hasattr(graph_def, 'as_graph_def'):
        graph_def = graph_def.as_graph_def()
    strip_def = strip_consts(graph_def, max_const_size=max_const_size)
    code = """
        <script>
          function load() {{
            document.getElementById("{id}").pbtxt = {data};
          }}
        </script>
        <link rel="import" href="https://tensorboard.appspot.com/tf-graph-basic.build.html" onload=load()>
        <div style="height:600px">
          <tf-graph-basic id="{id}"></tf-graph-basic>
        </div>
    """.format(data=repr(str(strip_def)), id='graph'+str(np.random.rand()))
  
    iframe = """
        <iframe seamless style="width:800px;height:620px;border:0" srcdoc="{}"></iframe>
    """.format(code.replace('"', '&quot;'))
    display(HTML(iframe))

show_graph(tf.get_default_graph().as_graph_def())

In [5]:
#running squareloss minimization
sess = tf.InteractiveSession()
tf.global_variables_initializer().run()
for ii in range(32000):
    ii=ii%4000
    batch_xs = ins[range((ii),(ii+1))]
    batch_ys = outs[range((ii),(ii+1))]
    sess.run(train_stepsquareloss, feed_dict={x: batch_xs, y_: batch_ys})

In [6]:

error = tf.reduce_mean(tf.cast(tf.square(y_-y2), tf.float64))
print(sess.run(error, feed_dict={x: ins, y_: outs}))

0.00011921040919


In [7]:
#running logloss minimization
sess = tf.InteractiveSession()
tf.global_variables_initializer().run()
for ii in range(32000):
    ii=ii%4000
    batch_xs = ins[range((ii),(ii+1))]
    batch_ys = outs[range((ii),(ii+1))]
    sess.run(train_steplogloss, feed_dict={x: batch_xs, y_: batch_ys})

In [8]:

error = tf.reduce_mean(tf.cast(tf.square(y_-y2), tf.float32))
print(sess.run(error, feed_dict={x: ins, y_: outs}))

2.78209e-07
